In [371]:
# (\__/)    Still a little messy, but this code was written for the Mobility for All Abilities
# (•ㅅ•)     on 9/21/2019 as part of the 'Reliable Access to Subways' team organized by
# / 　 づ    TransitCenter and United Equal Access NY.

In [408]:
import PyPDF2 
from tqdm import tqdm_notebook as tqdm
import re
import pandas
from tqdm import tqdm_notebook as tqdm

In [411]:
def convert_transit_pdf_to_csv(pdf, start, end):
    
    '''
    Input is a PDF file in the local directory, the first page
    of the PDF and the last page of the PDF. The function adjusts
    the first page to account for zero-based numbering. It will
    output a csv file back into the directory.
    
    There are a few issues with the code as-written. If an escalator
    or elevator is avialable 0.00% of the time, it will add an additional
    digit or character to the availability column. There is one other
    issue I've encountered where the subway lines aren't formatted properly.
    
    The comments will occaisonaly cut off and a fix for that is the first
    priority once this code is re-visited.
    '''

    page_range = list(range(start - 1,end))

    pdfFileObj = open(pdf, 'rb') 

    pdfReader = PyPDF2.PdfFileReader(pdfFileObj)

    lines = []
    availability = []
    units = []
    stations = []
    conditions = []

    condition = ''

    for page in page_range:
        pageObj = pdfReader.getPage(page) 
        current_page = pageObj.extractText().split('\n')

        for i in range(len(current_page[:-2])):
            
            # removes some titles that would otherwise be caught up in 
            if not re.match(string = current_page[i], pattern = '.* THAN 85% AVAILABILITY'):
                if len(current_page[i]) > 1:
                    if len(current_page[i]) > 40:
                        condition += current_page[i]
                    if re.match(string = current_page[i][-5:], pattern = '\d\.\d{2}\%'):
                        availability.append(current_page[i][-6:])
                        lines.append(current_page[i][:-6])
                    if re.match(string = current_page[i], pattern = 'E[LS]\d{3}'):
                        units.append(current_page[i])
                        stations.append(current_page[i + 1])
                        if len(condition) > 1:
                            conditions.append(condition)
                            condition = ''
                if i == len(current_page[:-2]) - 1:
                    conditions.append(condition)
                    condition = ''
    df_stations = pd.DataFrame(
        {'units': units,
         'stations': stations,
         'lines' : lines,
         'availability' : availability,
         'condition': conditions
        })

    df_stations.to_csv(pdf + ' converted.csv')

In [419]:
convert_transit_pdf_to_csv('180521_1030_Transit.pdf', 220, 222)

ValueError: arrays must all be same length

In [399]:
df_stations

,units,stations,lines,availability,condition
0,EL120,190 St,A,27.18%,This elevator was out multiple times due to le...
1,EL249,86 St,Q,84.06%,This elevator was out of service from 11/19/18...
2,ES104,Gun Hill Rd,25,83.76%,From 10/1/18 to 12/23/18 the escalator went ou...
3,ES105,Gun Hill Rd,25,83.48%,From 10/1/18 to 12/23/18 the escalator went ou...
4,ES106,West Farms Sq-E Tremont Av,25,79.51%,From 10/1/18 to 12/23/18 the escalator went ou...
5,ES108,Intervale Av,25,69.84%,The escalator was out service from 10/17/2018 ...
6,ES114,161 St-Yankee Stadium,4,69.45%,The escalator was out of service from 10/18/20...
7,ES120,Pelham Bay Park,6,22.72%,The escalator was out of service from 09/01/20...
8,ES305,Court St,R,50.95%,This escalator was out of service from 10/6/18...
9,ES347,Broadway Junction,ACJLZ,84.90%,This escalator was out of service 13 times due...


In [420]:
len(conditions)

27

In [421]:
len(lines)

27

In [422]:
len(units)

27

In [423]:
len(stations)

27

In [424]:
len(availability)

27

In [425]:
current_page

['Borough/ Unit',
 'Location',
 '24 Hr ',
 'Availability',
 'Comments',
 '4TH QUARTER 2018 ESCALATORS WITH LESS THAN 85% AVAILABILITY',
 'ES235',
 '34 St-Herald Sq ',
 'BDFM84.34%',
 'The escalator was out of service from 12/4/18 to 12/11/18 due to worn out handrail and countershaft chains ',
 'as well as defective brake assemblies. The countershaft assembly and chain were replaced and adjusted. ',
 'The right handrail chain was adjusted. The main brakes were replaced and adjusted as well as a controller ',
 '',
 'relay; the escalator was tested and returned to service. The escalator was out multiple times due to the ',
 '',
 'activation of various safety devices. Those safety devices were tested and adjusted as needed.',
 'ES255',
 'Grand Central-42 St ',
 '4567S82.56%',
 'This escalator was out of service from 11/30/18 to 12/3/18 due to an inspection of the proper operation of ',
 'the up and down operation. The escalator was found to be operating as designed and returned to service.